In [1]:
import pandas as pd 
from neo4j import GraphDatabase

In [4]:
class Neo4jConnection:
    def __init__(self,uri,user,password):
        self.driver = GraphDatabase.driver(uri,auth=(user,password))

    def clear_database(self):
        """
        DELETE all nodes and relationships from the database
        """
        query = "MATCH (n) DETACH DELETE n"
        result = self.execute_query(query)
        print("cleared all nodes and relationships from the databases")

    def close(self):
        self.driver.close()

    def execute_query(self,query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return [record for record in result]

In [5]:
## seif credentials
# uri = "neo4j+ssc://e45fa759.databases.neo4j.io"
# user = "neo4j"
# password = "ViO4jopeE4TjckK7VCChP6q6JOXn0DBVl5E8NhvMqv8"

#abushanab credentials
uri = "neo4j+ssc://b0d23506.databases.neo4j.io"
user = "neo4j"
password = "yA219NUjGwZeHjL7lQgGUHfzSJsQtneBlXqGByjik8c"

conn = Neo4jConnection(uri,user,password)
conn.clear_database()

cleared all nodes and relationships from the databases


In [2]:
df = pd.read_csv("fpl_two_seasons.csv")

In [7]:
data = df[['name', 'element', 'season', 'fixture', 'minutes',
               'goals_scored', 'assists', 'total_points', 'bonus',
               'clean_sheets', 'goals_conceded', 'own_goals',
               'penalties_saved', 'penalties_missed', 'yellow_cards',
               'red_cards', 'saves', 'bps', 'influence', 'creativity',
               'threat', 'ict_index', 'form']]
    



rel = data.drop(['name','element','season','fixture'], axis = 1)
rel = rel.to_dict('records')

rel

[{'minutes': 0,
  'goals_scored': 0,
  'assists': 0,
  'total_points': 0,
  'bonus': 0,
  'clean_sheets': 0,
  'goals_conceded': 0,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yellow_cards': 0,
  'red_cards': 0,
  'saves': 0,
  'bps': 0,
  'influence': 0.0,
  'creativity': 0.0,
  'threat': 0,
  'ict_index': 0.0,
  'form': 0.0},
 {'minutes': 45,
  'goals_scored': 0,
  'assists': 0,
  'total_points': 1,
  'bonus': 0,
  'clean_sheets': 0,
  'goals_conceded': 0,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yellow_cards': 0,
  'red_cards': 0,
  'saves': 0,
  'bps': -2,
  'influence': 0.0,
  'creativity': 1.1,
  'threat': 6,
  'ict_index': 0.5,
  'form': 0.0},
 {'minutes': 0,
  'goals_scored': 0,
  'assists': 0,
  'total_points': 0,
  'bonus': 0,
  'clean_sheets': 0,
  'goals_conceded': 0,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yellow_cards': 0,
  'red_cards': 0,
  'saves': 0,
  'bps': 0,
  'influence': 0.0,
  

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51952 entries, 0 to 51951
Data columns (total 35 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   season             51952 non-null  object 
 1   name               51952 non-null  object 
 2   position           51952 non-null  object 
 3   assists            51952 non-null  int64  
 4   bonus              51952 non-null  int64  
 5   bps                51952 non-null  int64  
 6   clean_sheets       51952 non-null  int64  
 7   creativity         51952 non-null  float64
 8   element            51952 non-null  int64  
 9   fixture            51952 non-null  int64  
 10  goals_conceded     51952 non-null  int64  
 11  goals_scored       51952 non-null  int64  
 12  ict_index          51952 non-null  float64
 13  influence          51952 non-null  float64
 14  kickoff_time       51952 non-null  object 
 15  minutes            51952 non-null  int64  
 16  own_goals          519

# Creating Nodes

1. Nodes: 
1. Season: [season_name] 
2. Gameweek: [season, GW_number] 
3. Fixture: [season, fixture_number], kickoff_time 
4. Team: [name] 
5. Player: [player_name, player_element] 
6. Position: [name]

## 1. Season: [season_name]

In [8]:
def createSeasons(conn, df):
    
    seasons = df['season'].unique().tolist()

    query = """
    UNWIND $seasons AS season_name
    MERGE (s: Season {season_name: season_name})
    RETURN count(s)
    """
    return conn.execute_query(query, {"seasons": seasons})

In [9]:
createSeasons(conn, df)

[<Record count(s)=2>]

## 2. Gameweek: [season, GW_number]


In [10]:
def createGW(conn, df):
    gameweek_data = df[['season', 'GW']].drop_duplicates()
    gameweeks = gameweek_data.to_dict('records')
    query = """
    UNWIND $gameweeks AS gw
    MERGE (g:Gameweek {season: gw.season, GW_number: gw.GW})
    RETURN count(g)
    """
    results = conn.execute_query(query, parameters={"gameweeks": gameweeks})
    print(f"Successfully processed {results} Gameweek nodes")

In [11]:
createGW(conn,df)

Successfully processed [<Record count(g)=75>] Gameweek nodes


## we realized that season 22-23 gw 7 is missing

## 3. Fixture: [season, fixture_number], kickoff _time

In [12]:
def createFixtures(conn,df):
    fixture_data = df[['season','fixture','kickoff_time']].drop_duplicates()
    fixtures = fixture_data.to_dict('records')

    query = """
    UNWIND $fixtures AS fixture
    MERGE (f:Fixture {season:fixture.season, fixture_number:fixture.fixture})
    SET f.kickoff_time = fixture.kickoff_time
    RETURN count(f)
    """

    result = conn.execute_query(query,parameters= {"fixtures": fixtures})
    print(f"Successfully created {result} fixture nodes")




In [13]:
createFixtures(conn,df)

Successfully created [<Record count(f)=760>] fixture nodes


## 4. Team: [name]



In [14]:
def createTeams(conn,df):
    teams_data = df[['home_team']].drop_duplicates()
    teams = teams_data.to_dict('records')

    query = """
    UNWIND $teams AS teams
    MERGE (t: Team {name: teams.home_team})
    RETURN count(t)
    """
    result = conn.execute_query(query, parameters={"teams": teams})
    print(f"Successfully executed {result} Team nodes")

    

In [15]:
createTeams(conn,df)

Successfully executed [<Record count(t)=23>] Team nodes


## 5.Player: [player_name, player_element]

In [16]:
def createPlayers(conn,df):
    players_data = df[['name', 'element']].drop_duplicates()
    players = players_data.to_dict('records')

    query = """
    UNWIND $players AS player
    MERGE (p: Player {player_name: player.name, player_element: player.element})
    RETURN count(p)
    """
    result = conn.execute_query(query, parameters={"players": players})
    print(f"Successfully created {result} Player nodes")


In [17]:
createPlayers(conn,df)

Successfully created [<Record count(p)=1513>] Player nodes


## 6.Position: [name]

In [18]:
def createPositions(conn,df):
    positions_data = df[['position']].drop_duplicates()
    positions = positions_data.to_dict('records')

    query = """
    UNWIND $positions AS position
    MERGE (p: Position {name: position.position})
    RETURN count(p)
    """
    result = conn.execute_query(query, parameters={"positions": positions})
    print(f"Successfully created {result} Positions nodes")


In [19]:
createPositions(conn,df)

Successfully created [<Record count(p)=4>] Positions nodes


# Creating Relations

1. Season: [season_name] 
2. Gameweek: [season, GW_number] 
3. Fixture: [season, fixture_number], kickoff_time 
4. Team: [name] 
5. Player: [player_name, player_element] 
6. Position: [name]

### 1.(Season[season_name]) - [:HAS_GW]-> (Gameweek[season, GW_number])

In [20]:
def create_has_gw_relation(conn,df):
    data = df[['season','GW']].drop_duplicates()
    rel = data.to_dict('records')

    query = """
    UNWIND $rel AS rel
    MATCH (s: Season {season_name: rel.season})
    MATCH (g: Gameweek {season: rel.season, GW_number: rel.GW})
    MERGE (s) - [r:HAS_GW] -> (g)
    RETURN count(r) as relations_created
    """

    result = conn.execute_query(query, parameters={"rel":rel})
    print(f"Successfully created {result[0]['relations_created']} :HAS_GW relation")

In [21]:
create_has_gw_relation(conn,df)

Successfully created 75 :HAS_GW relation


### 2. (Gameweek) - [:HAS_FIXTURE]-> (Fixture)

In [22]:
def create_has_fixture_relation(conn, df):
    data = df[['season','fixture','GW']].drop_duplicates()
    rel = data.to_dict('records')
    
    query = """
    UNWIND $rel AS rel
    MATCH (g:Gameweek {season: rel.season, GW_number: rel.GW})
    MATCH (f:Fixture {season: rel.season, fixture_number: rel.fixture})
    MERGE (g)-[r:HAS_FIXTURE]->(f)
    RETURN count(r) as relations_created
    """
    
    result = conn.execute_query(query, parameters={"rel": rel})
    count = result[0]['relations_created']
    print(f"Created {count} HAS_FIXTURE relationships")
    return count

In [23]:
create_has_fixture_relation(conn,df)

Created 760 HAS_FIXTURE relationships


760

### 3.(Fixture) - [:HAS_HOME_TEAM]-> (Team)

In [24]:
def create_home_team_relation(conn, df):
    data = df[['season','fixture','home_team']].drop_duplicates()
    rel = data.to_dict('records')
    
    query = """
    UNWIND $rel AS rel
    MATCH (f:Fixture {season: rel.season, fixture_number: rel.fixture})
    MATCH (t:Team {name: rel.home_team})
    MERGE (f)-[r:HAS_HOME_TEAM]->(t)
    RETURN count(r) as relations_created
    """
    
    result = conn.execute_query(query, parameters={"rel": rel})
    count = result[0]['relations_created']
    print(f"Created {count} HAS_HOME_TEAM relationships")
    return count

In [25]:
create_home_team_relation(conn,df)

Created 760 HAS_HOME_TEAM relationships


760

### 4.(Fixture) - [:HAS_AWAY_TEAM]-> (Team)

In [26]:
def create_away_team_relation(conn, df):
    data = df[['season','fixture','away_team']].drop_duplicates()
    rel = data.to_dict('records')
    
    query = """
    UNWIND $rel AS rel
    MATCH (f:Fixture {season: rel.season, fixture_number: rel.fixture})
    MATCH (t:Team {name: rel.away_team})
    MERGE (f)-[r:HAS_AWAY_TEAM]->(t)
    RETURN count(r) as relations_created
    """
    
    result = conn.execute_query(query, parameters={"rel": rel})
    count = result[0]['relations_created']
    print(f"Created {count} HAS_AWAY_TEAM relationships")
    return count

In [27]:
create_away_team_relation(conn,df)

Created 760 HAS_AWAY_TEAM relationships


760

### 5.(Player) - [:PLAYS_AS]-> (Position)

In [28]:
def create_plays_as_relation(conn, df):
    data = df[['name', 'element','position']].drop_duplicates()
    rel = data.to_dict('records')
    
    query = """
    UNWIND $rel AS rel
    MATCH (p:Player {player_name: rel.name, player_element: rel.element})
    MATCH (pos:Position {name: rel.position})
    MERGE (p)-[r:PLAYS_AS]->(pos)
    RETURN count(r) as relations_created
    """
    
    result = conn.execute_query(query, parameters={"rel": rel})
    count = result[0]['relations_created']
    print(f"Created {count} PLAYS_AS relationships")
    return count

In [29]:
create_plays_as_relation(conn,df)

Created 1518 PLAYS_AS relationships


1518

### 6.(Player) - [:PLAYED_IN]-> (Fixture)

In [ ]:
# def create_played_in_relation(conn, df):
#     # Select all the performance properties for the relationship
#     data = df[['name', 'element', 'season', 'fixture', 'minutes', 'goals_scored', 'assists', 
#                'total_points', 'bonus', 'clean_sheets', 'goals_conceded', 'own_goals',
#                'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 
#                'saves', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'form']]
#     rel = data.to_dict('records')
    
#     query = """
# UNWIND $rel AS rel
# MATCH (p:Player {player_name: rel.name, player_element: rel.element})
# MATCH (f:Fixture {season: rel.season, fixture_number: rel.fixture})
# MERGE (p)-[r:PLAYED_IN]->(f)
# SET r += {
#     minutes: rel.minutes,
#     goals_scored: rel.goals_scored,
#     assists: rel.assists,
#     total_points: rel.total_points,
#     bonus: rel.bonus,
#     clean_sheets: rel.clean_sheets,
#     goals_conceded: rel.goals_conceded,
#     own_goals: rel.own_goals,
#     penalties_saved: rel.penalties_saved,
#     penalties_missed: rel.penalties_missed,
#     yellow_cards: rel.yellow_cards,
#     red_cards: rel.red_cards,
#     saves: rel.saves,
#     bps: rel.bps,
#     influence: rel.influence,
#     creativity: rel.creativity,
#     threat: rel.threat,
#     ict_index: rel.ict_index,
#     form: rel.form
# }
# RETURN count(r) AS relations_created;

#     """
    
#     result = conn.execute_query(query, parameters={"rel": rel})
#     count = result[0]['relations_created']
#     print(f"Created {count} PLAYED_IN relationships with performance data")
#     return count

# def create_played_in_relation(conn, df, batch_size=2000):
#     data = df[['name', 'element', 'season', 'fixture', 'minutes',
#                'goals_scored', 'assists', 'total_points', 'bonus',
#                'clean_sheets', 'goals_conceded', 'own_goals',
#                'penalties_saved', 'penalties_missed', 'yellow_cards',
#                'red_cards', 'saves', 'bps', 'influence', 'creativity',
#                'threat', 'ict_index', 'form']]

#     rel = data.to_dict('records')

#     query = """
#     UNWIND $batch AS rel
#     MATCH (p:Player {player_name: rel.name, player_element: rel.element})
#     MATCH (f:Fixture {season: rel.season, fixture_number: rel.fixture})
#     MERGE (p)-[r:PLAYED_IN]->(f)
#     SET r += rel
#     """

#     total = 0
#     for i in range(0, len(rel), batch_size):
#         batch = rel[i:i+batch_size]
#         conn.execute_query(query, parameters={"batch": batch})
#         total += len(batch)
#         print(f"Inserted {total}/{len(rel)}")

#     print("DONE")
#     return total


def PlayerPlayedInFixture(tx, rows):
    query = """
    UNWIND $rows AS row
    MATCH (p:Player {player_name: row.name, player_element: row.element})
    MATCH (f:Fixture {season: row.season, fixture_number: row.fixture})

    MERGE (p)-[r:PLAYED_IN]->(f)
    SET r.minutes = toInteger(row.minutes),
        r.goals_scored = toInteger(row.goals_scored),
        r.assists = toInteger(row.assists),
        r.total_points = toInteger(row.total_points),
        r.bonus = toInteger(row.bonus),
        r.clean_sheets = toInteger(row.clean_sheets),
        r.goals_conceded = toInteger(row.goals_conceded),
        r.own_goals = toInteger(row.own_goals),
        r.penalties_saved = toInteger(row.penalties_saved),
        r.penalties_missed = toInteger(row.penalties_missed),
        r.yellow_cards = toInteger(row.yellow_cards),
        r.red_cards = toInteger(row.red_cards),
        r.saves = toInteger(row.saves),
        r.bps = toInteger(row.bps),
        r.influence = toFloat(row.influence),
        r.creativity = toFloat(row.creativity),
        r.threat = toFloat(row.threat),
        r.ict_index = toFloat(row.ict_index),
        r.form = toFloat(row.form);
    """

    tx.run(query, rows=rows)

playedInDF = df[df["minutes"] > 0][[
"name", "element", "season", "fixture",

"minutes",
"goals_scored",
"assists",
"total_points",
"bonus",
"clean_sheets",
"goals_conceded",
"own_goals",
"penalties_saved",
"penalties_missed",
"yellow_cards",
"red_cards",
"saves",
"bps",
"influence",
"creativity",
"threat",
"ict_index",
"form"
]]
playedInDF

rows = playedInDF.to_dict("records")

In [ ]:
with conn.driver.session() as session:
    session.execute_write(PlayerPlayedInFixture, rows)

In [33]:
# create_played_in_relation(conn,df)

# Queries

## 1. How many gameweeks were played during the seasons provided? 
### Answer: 75

In [34]:
query1 = """
MATCH (g:Gameweek)
RETURN count(g) as gameweeks
""" 

result = conn.execute_query(query1)
print(result[0]['gameweeks'])

75


## 2. How many player nodes were created (note: the same player can have different nodes representing him in each season) 
### Answer: 1513 

In [35]:
query2 = """
MATCH (p:Player)
RETURN count(p) as players
"""
result = conn.execute_query(query2)
print(result[0]['players'])

1513


In [36]:
query3 = """
MATCH (s1:Season {season_name: '2021-22'})-[:HAS_GW]->(:Gameweek)-[:HAS_FIXTURE]->(f:Fixture)-[:HAS_HOME_TEAM|HAS_AWAY_TEAM]->(t:Team)
WITH collect(DISTINCT t.name) as teams_2021_22

MATCH (s2:Season {season_name: '2022-23'})-[:HAS_GW]->(:Gameweek)-[:HAS_FIXTURE]->(f:Fixture)-[:HAS_HOME_TEAM|HAS_AWAY_TEAM]->(t:Team)
WITH teams_2021_22, collect(DISTINCT t.name) as teams_2022_23

RETURN [team IN teams_2021_22 WHERE NOT team IN teams_2022_23] as qualified_only_in_2021_22"""
result = conn.execute_query(query3)
print(result[0]['qualified_only_in_2021_22'])

['Burnley', 'Watford', 'Norwich']


In [37]:
query4 = """
MATCH (s:Season {season_name: '2022-23'})-[:HAS_GW]->(gw:Gameweek)-[:HAS_FIXTURE]->(f:Fixture)
MATCH (p:Player)-[played:PLAYED_IN]->(f)
RETURN p.player_name as player_name, played.total_points as points, gw.GW_number as gameweek
ORDER BY played.total_points DESC
LIMIT 1
"""
result = conn.execute_query(query4)
record = result[0]

print(f"{record['points']} points by {record['player_name']}")

23 points by Erling Haaland


In [38]:
# First, find all possible Mohamed Elneny variations
query = """
MATCH (p:Player)
WHERE toLower(p.player_name) CONTAINS 'elneny'
RETURN p.player_name, p.player_element 
"""
result = conn.execute_query(query)

print(f"Found {len(result)} variations of Mohamed Elneny:")
for i, record in enumerate(result, 1):
    print(i , record)
    # print(f"{i}. Name: {record['player_name']}, Element: {record['player_element']}")

Found 2 variations of Mohamed Elneny:
1 <Record p.player_name='Mohamed Elneny' p.player_element=4>
2 <Record p.player_name='Mohamed Naser El Sayed Elneny' p.player_element=12>


In [39]:
query5 = """
// Count matches for ALL Mohamed Elneny variations
MATCH (p:Player)
WHERE toLower(p.player_name) CONTAINS 'elneny'
MATCH (p)-[played:PLAYED_IN]->(f:Fixture)
WHERE played.minutes > 0
RETURN count(played) as total_matches_played
"""
result = conn.execute_query(query5)
record = result[0]

print(result[0]['total_matches_played'])

19


In [40]:
query_scoring = """
// rule.txt - Complete scoring rule implementation
MATCH (p:Player)-[:PLAYS_AS]->(pos:Position)
WITH p, collect(pos.name) as positions
WHERE size(positions) > 1 AND 'DEF' IN positions

WITH p, positions
MATCH (p)-[played:PLAYED_IN]->(f:Fixture)
MATCH (f)<-[:HAS_FIXTURE]-(gw:Gameweek)-[:HAS_GW]-(s:Season)

// Calculate what the score SHOULD have been if player was defender
WITH p, positions, played, f, s, gw,
     // General scoring rules
     played.assists * 3 +
     CASE 
         WHEN played.minutes < 60 THEN 1
         WHEN played.minutes >= 60 THEN 2
         ELSE 0 
     END +
     COALESCE(played.penalties_missed, 0) * -2 +
     COALESCE(played.own_goals, 0) * -2 +
     COALESCE(played.yellow_cards, 0) * -1 +
     COALESCE(played.red_cards, 0) * -3 +
     
     // Defender-specific scoring
     COALESCE(played.goals_scored, 0) * 6 +
     CASE 
         WHEN played.minutes > 60 AND COALESCE(played.clean_sheets, 0) = 1 THEN 4
         ELSE 0 
     END +
     FLOOR(COALESCE(played.goals_conceded, 0) / 2.0) * -1 +
     // Penalty saves and saves don't apply to defenders (0 points)
     0 as def_score,

     // Store fixture details
     s.season_name as season,
     f.fixture_number as fixture_number,
     played.total_points as total_points

// Check if defender score matches actual points (with small tolerance for floating point)
WITH p, 
     size([played IN [(p)-[played:PLAYED_IN]->(f) | played] WHERE played.minutes > 0]) as total_fixtures,
     collect({
         def_score: def_score,
         season: season,
         fixture_number: fixture_number,
         matches_def: abs(def_score - total_points) < 0.1, // Small tolerance for comparison
         total_points: total_points
     }) as all_fixtures

// Filter only fixtures where defender score doesn't match actual points
WITH p, 
     total_fixtures,
     [fixture IN all_fixtures WHERE NOT fixture.matches_def] as incorrect_fixtures

WHERE size(incorrect_fixtures) > 0
RETURN p.player_name as player,
       size(incorrect_fixtures) as not_def_fixtures,
       total_fixtures,
       incorrect_fixtures as fixture_details
ORDER BY not_def_fixtures DESC
"""

result = conn.execute_query(query_scoring)

# Format the output exactly as specified
print("player\tnot_def_fixtures\ttotal_fixtures\tfixture_details")
print("-" * 80)

for record in result:
    player = record['player']
    not_def_fixtures = record['not_def_fixtures']
    total_fixtures = record['total_fixtures']
    fixture_details = record['fixture_details']
    
    # Print in the exact format with proper spacing
    print(f'"{player}"\t{not_def_fixtures}\t{total_fixtures}')
    print("[")
    
    for i, fixture in enumerate(fixture_details):
        print("  {")
        print(f'    "def_score": {fixture["def_score"]},')
        print(f'    "season": "{fixture["season"]}",')
        print(f'    "fixture_number": {fixture["fixture_number"]},')
        print(f'    "matches_def": {str(fixture["matches_def"]).lower()},')
        print(f'    "total_points": {fixture["total_points"]}')
        print("  }" + ("," if i < len(fixture_details) - 1 else ""))
    
    print("]")
    print()  # Add space between players

player	not_def_fixtures	total_fixtures	fixture_details
--------------------------------------------------------------------------------
"Alexandre Lacazette"	15	1
[
  {
    "def_score": 1.0,
    "season": "2021-22",
    "fixture_number": 311,
    "matches_def": false,
    "total_points": 2
  },
  {
    "def_score": 7.0,
    "season": "2021-22",
    "fixture_number": 161,
    "matches_def": false,
    "total_points": 3
  },
  {
    "def_score": 6.0,
    "season": "2021-22",
    "fixture_number": 221,
    "matches_def": false,
    "total_points": 2
  },
  {
    "def_score": 7.0,
    "season": "2021-22",
    "fixture_number": 71,
    "matches_def": false,
    "total_points": 5
  },
  {
    "def_score": 6.0,
    "season": "2021-22",
    "fixture_number": 81,
    "matches_def": false,
    "total_points": 2
  },
  {
    "def_score": 6.0,
    "season": "2021-22",
    "fixture_number": 101,
    "matches_def": false,
    "total_points": 2
  },
  {
    "def_score": 1.0,
    "season": "2021-22",
